Step 1: Add libraries such as TensorFlow.
TensorFlow will be our main framework for developing and optimizing the autoencoder. Among the other tools we’ll be utilizing for data processing and visualization are pandas, matplotlib, and numpy.

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Step 2: Load and prepare the dataset
TensorFlow provides the Fashion MNIST dataset, which we will utilize. The training and testing sets will be loaded and the pixel values will be normalized to the interval [-1, 1]. Since our autoencoder uses a single neuron for the output layer. We will also restructure the pictures to have a single channel.

In [ ]:
# Load the dataset
(x_train, _), (x_test, _) = tf.keras.datasets.fashion_mnist.load_data()

# Normalize the pixel values
x_train = x_train.astype('float32') / 255.0 * 2 - 1
x_test = x_test.astype('float32') / 255.0 * 2 - 1

# Reshape the images to have a single channel
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

Step 3: Define the model architecture and parameters
Two thick layers—one for the encoder and one for the decoder—will be used in our straightforward autoencoder. 64 neurons make up the encoder, which maps the input to a 64-dimensional latent space using a sigmoid activation function. In order to map the latent space back to the original picture space of 784 dimensions (28×28 pixels). The decoder will employ a tan h activation function with 784 neurons.

Model definition will be done via the Keras Sequential API and model compilation will be done using an Adam optimizer and a mean squared error loss function.

In [ ]:
# Define the model architecture
model = tf.keras.Sequential([
  # Encoder layer
  tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
  tf.keras.layers.Dense(64, activation='sigmoid'),
  # Decoder layer
  tf.keras.layers.Dense(784, activation='tanh'),
  tf.keras.layers.Reshape((28, 28, 1))
])

# Compile the model
model.compile(loss='mse', optimizer='adam')

Step 4: Train and evaluate the model
Using a batch size of 256 and a validation split of 0.2. We will train the model across ten epochs. After every epoch, the model weights will be saved using a callback method. In order to assess the model we will compute the reconstruction error on the test set and display a few of the original and rebuilt photos.

In [ ]:
# Define a callback function to save the model weights
class SaveModel(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        model.save_weights(f'autoencoder_{epoch+1}.weights.h5')
        print(f'Saved model weights for epoch {epoch+1}')

# Train the model
model.fit(x_train, x_train, epochs=10, batch_size=256, validation_split=0.2, callbacks=[SaveModel()])

# Evaluate the model
test_loss = model.evaluate(x_test, x_test)
print(f'Test loss: {test_loss}')

# Visualize some of the original and reconstructed images
n = 10  # Number of images to display
plt.figure(figsize=(20, 4))
for i in range(n):
    # Display original image
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28), cmap='gray')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display reconstructed image
    ax = plt.subplot(2, n, i + 1 + n)
    reconstructed = model.predict(x_test[i].reshape(1, 28, 28, 1))
    plt.imshow(reconstructed.reshape(28, 28), cmap='gray')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

Building Image denoising
The process of removing noise from a picture, which might include many kinds of such speckle, salt-and-pepper, and Gaussian noise, is known as image denoising. Image denoising can enhance an image clarity and quality or get it ready for other uses, such segmentation or classification.

We will use a neural autoencoder to denoise the same MNIST dataset in this case. The autoencoder will be taught to separate the clean pictures from the noisy ones once the original photographs are exposed to Gaussian noise.

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Step 2: Load and prepare the dataset

In [ ]:
# Load the dataset
(x_train, _), (x_test, _) = tf.keras.datasets.fashion_mnist.load_data()

# Normalize the pixel values
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Reshape the images to have a single channel
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

# Add Gaussian noise to the images
noise_factor = 0.1
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)

# Clip the noisy images to the range [0, 1]
x_train_noisy = np.clip(x_train_noisy, 0.0, 1.0)
x_test_noisy = np.clip(x_test_noisy, 0.0, 1.0)

Step 3: Define the model architecture and parameters
Four convolutional layers—two for the encoder and two for the decoder—will be used in our convolutional autoencoder. By using 32 and 16 filters, together with a kernel size of 3 and a stride of 2. The encoder will cut the input’s spatial dimensions in half. The input spatial dimensions will be increased by half thanks to the decoder employment of 16 and 1 filters, a kernel size of 3, and a stride of 2. A sigmoid activation function will also be used by the decoder to map the output to the range [0, 1].

In [ ]:
# Define the encoder
def encoder(inputs):
  # Convolutional layer with 32 filters, 3x3 kernel, and 2x2 stride
  x = tf.keras.layers.Conv2D(32, 3, 2, padding='same')(inputs)
  # Convolutional layer with 16 filters, 3x3 kernel, and 2x2 stride
  x = tf.keras.layers.Conv2D(16, 3, 2, padding='same')(x)
  return x

# Define the decoder
def decoder(inputs):
  # Convolutional transpose layer with 16 filters, 3x3 kernel, and 2x2 stride
  x = tf.keras.layers.Conv2DTranspose(16, 3, 2, padding='same')(inputs)
  # Convolutional transpose layer with 1 filter, 3x3 kernel, and 2x2 stride
  x = tf.keras.layers.Conv2DTranspose(1, 3, 2, padding='same', activation='sigmoid')(x)
  return x

# Define the input layer
inputs = tf.keras.layers.Input(shape=(28, 28, 1))

# Define the model
model = tf.keras.Model(inputs=inputs, outputs=decoder(encoder(inputs)))

# Compile the model
model.compile(loss='mse', optimizer='adam')

Step 4: Train and evaluate the model
Using a batch size of 128 and a validation split of 0.2. We will train the model for 50 epochs. After every epoch, the model weights will be saved using a callback method. In order to assess the model , we will compute the reconstruction error on the test set and visualize a portion of the noisy, original photos that have been denoised.

In [ ]:
# Define a callback function to save the model weights
class SaveModel(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        model.save_weights(f'denoising_{epoch+1}.weights.h5')
        print(f'Saved model weights for epoch {epoch+1}')

# Train the model
model.fit(x_train_noisy, x_train, epochs=50, batch_size=128, validation_split=0.2, callbacks=[SaveModel()])

# Evaluate the model
test_loss = model.evaluate(x_test_noisy, x_test)
print(f'Test loss: {test_loss}')

# Visualize some of the original and reconstructed images
n = 10  # Number of images to display
plt.figure(figsize=(20, 4))
for i in range(n):
    # Display original image
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28), cmap='gray')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display reconstructed image
    ax = plt.subplot(2, n, i + 1 + n)
    reconstructed = model.predict(x_test[i].reshape(1, 28, 28, 1))
    plt.imshow(reconstructed.reshape(28, 28), cmap='gray')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

conclusion
We have learnt how to utilize TensorFlow for generative modeling in this article. We have also seen two example code with step by step implementation using TensorFlow for generative modeling. This is just a glimpse into the exciting world of generative modeling with TensorFlow. With practice and exploration, you can use this powerful tool to bring your creative ideas to life from generating futuristic landscapes to composing new musical styles.